<a href="https://colab.research.google.com/github/raymondwcs/learning_spacy/blob/main/spaCy_training_(command_line%2C_Chinese).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy==3.0.6
!python -m spacy download zh_core_web_lg
!pip install -U pkuseg

2021-08-11 06:39:19.405941: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 605.1 MB 2.9 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_lg')


In [2]:
# https://spacy.io/usage/training#config
!python -m spacy init fill-config base_config.cfg config.cfg


2021-08-11 06:40:08.689481: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [6]:
TRAIN_DATA = [
  (
    '現時恒指已跌穿重要支持位26500點來看，港股繼續尋底的機會是頗高的。',
    {
        'entities':
        [
         (2,4,'FIN'),(9,12,'FIN'),(21,23,'FIN'),(25,27,'FIN')
        ]
    }
  ),
  (
    '恆生指數和創業板指數。',
    {
      'entities': 
      [
        (0,4,'FIN'),(5,10,'FIN')          
      ]
    }
  ),  
  (
    '中電、匯控、和黃8月暴升8.5%。',
    {
      'entities': 
      [
        (0,2,'STOCK'),(3,5,'STOCK'),(6,8,'STOCK'),(10,12,'FIN')         
      ]
    }
  ),
  (
    '恆指小雙底9月盼反彈。中電、匯控、和黃8月齊齊跌8.5%金融海嘯後最差。',
    {
      'entities': 
      [
        (0,2,'FIN'),(2,5,'FIN'),(8,10,'FIN'),(11,13,'STOCK'),(14,16,'STOCK'),(17,19,'STOCK')      
      ]
    }
  ),     
  (
    '匯控(00005)將於下周一（2日）公布2021年度中期業績。',
    {
        'entities':
        [
          (0,2,'STOCK'),(26,30,'FIN')
        ]
    }
  ),
  (
    '查看香港最新的股市報價、記錄、新聞及其他重要資訊，助你賣買股票及投資。',
   {
      'entities':
      [
        (2,4,'GPE'),(7,9,'FIN'),(9,11,'FIN')
      ]
   }
  ),
  (
    '匯控低開4%。港股收市跌。',
    {
        'entities':
        [
         (0,2,'STOCK'),(7,9,'FIN')
        ]
    }
  ),
  (
    '港股高開173點。',
   {
       'entities': [(0,2,'FIN'),(2,4,'FIN')]
   }
  ),
  (
    '報道引述匯控內部電郵稱',{'entities':[(4,6,'STOCK')]}
  ),
  (
    '匯控將在第四季開始進行股份回購',{'entities':[(0,2,'STOCK')]}
  ),
    (
    '花旗發表研究報告，下調匯控（0005）每股盈利預測',{'entities':[(11,13,'STOCK')]}
  ),
    (
    '匯控將恢復派息。',{'entities':[(0,2,'STOCK'),(5,7,'FIN')]}
  )
]

import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

# nlp = space.blank("zh") # load a new spacy model
nlp = spacy.load("zh_core_web_lg")

db = DocBin() # create a DocBin object

# https://spacy.io/usage/models
nlp.tokenizer.pkuseg_update_user_dict(['匯控','和黄','中電','派息','股份回購'])

for text, annot in tqdm(TRAIN_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label)
        if span is None:
            pass
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./train.spacy") # save the docbin object

100%|██████████| 12/12 [00:00<00:00, 173.90it/s]


In [7]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy --gpu-id 0

2021-08-11 07:10:37.754855: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2021-08-11 07:10:39,584] [INFO] Set up nlp object from config
[2021-08-11 07:10:39,688] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-08-11 07:10:39,693] [INFO] Created vocabulary
[2021-08-11 07:10:43,373] [INFO] Added vectors: zh_core_web_lg
[2021-08-11 07:10:43,373] [INFO] Finished initializing nlp object
[2021-08-11 07:10:49,247] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     56.71   13.61    8.55   33.33    0.14
108 

In [8]:
text = '匯控下個月派息'

# Now, try the trained model
# Load the saved model
nlp1 = spacy.load("./output/model-best") #load the best model

doc = nlp1(text) # input sample text

for token in doc:
  print(token.text, token.pos_, token.dep_)

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

匯控  
下  
個月  
派息  
